In [1]:
import os
import zipfile
import pandas as pd
import basedosdados as bd

INPUT = os.path.join(os.getcwd(), "input")
OUTPUT = os.path.join(os.getcwd(), "output")

os.makedirs(INPUT, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)

In [2]:
def read_sheet(sheet_name: str, skiprows: int = 2) -> pd.DataFrame:
    return pd.read_excel(
        os.path.join(
            INPUT,
            "2020_2022_INDIC_ED_ESP.xlsx"
        ),
        skiprows=skiprows,
        sheet_name=sheet_name,
        dtype=str
    )

In [3]:
sheets_etapa_ensino_serie = {
    "INDICADOR 2":"regente_aee",
    "INDICADOR 3":"aee",
    "INDICADOR 4":"regente_formacaocontinuada",
    "INDICADOR 5":"aee_formacaocontinuada",    
}

In [4]:
dfs = {
    name: read_sheet(sheet_name)
    for sheet_name, name in sheets_etapa_ensino_serie.items()
}

In [5]:
dataframes = {}

for table_name, columns in dfs.items():
    df = pd.DataFrame(columns)  # Create DataFrame for each table
    dataframes[table_name] = df  # Store the DataFrame in a dictionary

In [6]:
dataframes['aee'].columns

Index(['NU_ANO_CENSO', 'CO_REGIAO', 'NO_REGIAO', 'CO_UF', 'NO_UF', 'SIGLA',
       'NO_MUNICIPIO', 'CO_MUNICIPIO', 'DOCENTES'],
      dtype='object')

In [7]:
RENAMES_COLUMNS = {
    "regente_aee": {
        'NU_ANO_CENSO':'ano', 
        #'CO_REGIAO',
        #'NO_REGIAO', 
        #'CO_UF', 
        #'NO_UF', 
        'SIGLA':"sigla_uf",
        #'NO_MUNICIPIO', 
        'CO_MUNICIPIO':"id_municipio", 
        'DOCENTES':'quantidade_docente_regente_aee'
    },
    "aee": {
        'NU_ANO_CENSO':'ano', 
        #'CO_REGIAO',
        #'NO_REGIAO', 
        #'CO_UF', 
        #'NO_UF', 
        'SIGLA':"sigla_uf",
        #'NO_MUNICIPIO', 
        'CO_MUNICIPIO':"id_municipio", 
        'DOCENTES':'quantidade_docente_aee'
    },
    "regente_formacaocontinuada": {
        'NU_ANO_CENSO':'ano', 
        #'CO_REGIAO',
        #'NO_REGIAO', 
        #'CO_UF', 
        #'NO_UF', 
        'SIGLA':"sigla_uf",
        #'NO_MUNICIPIO', 
        'CO_MUNICIPIO':"id_municipio", 
        'DOCENTES':'quantidade_docente_regente_formacao_continuada'
    },
    "aee_formacaocontinuada": {
        'NU_ANO_CENSO':'ano', 
        #'CO_REGIAO',
        #'NO_REGIAO', 
        #'CO_UF', 
        #'NO_UF', 
        'SIGLA':"sigla_uf",
        #'NO_MUNICIPIO', 
        'CO_MUNICIPIO':"id_municipio", 
        'DOCENTES':'quantidade_docente_aee_formacao_continuada'
    }
}

In [8]:
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols_drop = [
        col
        for col in df.columns
        if col.startswith("CO_")
        or col.startswith("NO_")
    ]

    return df.drop(columns=cols_drop)

dfs = {
    name: drop_unused_columns(
        df.rename(columns=RENAMES_COLUMNS[name], errors="raise")
    )
    for name, df in dfs.items()
}

In [9]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()

Sheet: regente_aee
Index(['ano', 'sigla_uf', 'id_municipio', 'quantidade_docente_regente_aee'], dtype='object')

Sheet: aee
Index(['ano', 'sigla_uf', 'id_municipio', 'quantidade_docente_aee'], dtype='object')

Sheet: regente_formacaocontinuada
Index(['ano', 'sigla_uf', 'id_municipio',
       'quantidade_docente_regente_formacao_continuada'],
      dtype='object')

Sheet: aee_formacaocontinuada
Index(['ano', 'sigla_uf', 'id_municipio',
       'quantidade_docente_aee_formacao_continuada'],
      dtype='object')



In [10]:
def remove_nulls(df: pd.DataFrame, name: str) -> pd.DataFrame:
    # Ensure you're working with the whole DataFrame
    df = df.dropna(subset=['id_municipio'])  # Remove rows where 'id_municipio' is NaN
    df = df.loc[df['id_municipio'].astype(str) != " "]  # Filter out rows where 'id_municipio' is empty space
    return df

# Apply remove_nulls to each DataFrame in the dictionary
dfs = {
    name: remove_nulls(df, name)  # This keeps the original key (sheet name) with the cleaned DataFrame
    for name, df in dfs.items()
}

In [11]:
dfs

{'regente_aee':         ano sigla_uf id_municipio quantidade_docente_regente_aee
 3      2020       RO      1100015                            291
 4      2020       RO      1100379                            128
 5      2020       RO      1100403                            135
 6      2020       RO      1100346                            189
 7      2020       RO      1100023                           1018
 ...     ...      ...          ...                            ...
 16803  2022       GO      5222302                             78
 16804  2022       GO      5200175                             58
 16805  2022       GO      5200209                             31
 16806  2022       GO      5200258                           1818
 16808  2022       DF      5300108                          30821
 
 [16710 rows x 4 columns],
 'aee':         ano sigla_uf id_municipio quantidade_docente_aee
 3      2020       RO      1100015                     10
 4      2020       RO      1100379       

In [12]:
import pandas as pd

# Assuming 'dfs' is your dictionary with four tables
df_regente_aee = pd.DataFrame(dfs['regente_aee'])
df_aee = pd.DataFrame(dfs['aee'])
df_regente_formacaocontinuada = pd.DataFrame(dfs['regente_formacaocontinuada'])  # Example third table
df_aee_formacaocontinuada = pd.DataFrame(dfs['aee_formacaocontinuada'])  # Example fourth table

# Merge all four DataFrames with outer join on 'ano', 'sigla_uf', 'id_municipio'
merged_df = df_regente_aee.merge(df_aee, on=['ano', 'sigla_uf', 'id_municipio'], how='outer')
merged_df = merged_df.merge(df_regente_formacaocontinuada, on=['ano', 'sigla_uf', 'id_municipio'], how='outer')
merged_df = merged_df.merge(df_aee_formacaocontinuada, on=['ano', 'sigla_uf', 'id_municipio'], how='outer')


In [13]:
merged_df['quantidade_docente_regente_aee'] = pd.to_numeric(merged_df['quantidade_docente_regente_aee'], errors='coerce')
merged_df['quantidade_docente_aee'] = pd.to_numeric(merged_df['quantidade_docente_aee'], errors='coerce')

In [14]:
merged_df['quantidade_docente_regente'] = merged_df['quantidade_docente_regente_aee'] - merged_df['quantidade_docente_aee']

In [15]:
merged_df = merged_df.drop(columns='quantidade_docente_regente_aee', axis='')

In [16]:
merged_df = merged_df[[
    'ano', 
    'sigla_uf', 
    'id_municipio',
    'quantidade_docente_regente',
    'quantidade_docente_aee',
    'quantidade_docente_regente_formacao_continuada',
    'quantidade_docente_aee_formacao_continuada']]

In [17]:
merged_df

,ano,sigla_uf,id_municipio,quantidade_docente_regente,quantidade_docente_aee,quantidade_docente_regente_formacao_continuada,quantidade_docente_aee_formacao_continuada
0,2020,RO,1100015,281.0,10.0,14,5
1,2020,RO,1100379,125.0,3.0,1,NaN
2,2020,RO,1100403,131.0,4.0,2,NaN
3,2020,RO,1100346,178.0,11.0,9,1
4,2020,RO,1100023,963.0,55.0,34,15
...,...,...,...,...,...,...,...
16705,2022,GO,5222302,75.0,3.0,2,NaN
16706,2022,GO,5200175,NaN,NaN,3,NaN
16707,2022,GO,5200209,29.0,2.0,6,1
16708,2022,GO,5200258,1760.0,58.0,310,39


In [18]:
path = os.path.join(
        OUTPUT, "educacao_especial_aee_docente"
    )

os.makedirs(path, exist_ok=True)
merged_df.astype(str).to_csv(os.path.join(path, "data.csv"), index=False)